In [1]:
import tensorflow as tf
import tflearn

In [2]:
import numpy as np

In [3]:
sess = tf.Session()
optim = tf.train.AdamOptimizer(learning_rate=0.001)

In [4]:
from tqdm import tqdm

In [5]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [6]:
import BayesianNN

In [7]:
bnn = BayesianNN.BayesianFC([1,256,256,256,1])

In [8]:
def test_func(x):
    e = np.random.normal(0,0.01)
    return x + 0.3*np.sin(2 *np.pi * (x + e)) + 0.3*np.sin(4 * np.pi*(x + e)) + e

In [9]:
data = [(x, test_func(x)) for x in np.linspace(0,0.5,1000)]

In [10]:
data_input = list(map(lambda x: [x[0]], data))
data_target = list(map(lambda x: [x[1]], data))

In [11]:
def normal_nn(optim):
    data_input = tf.placeholder(tf.float32, shape=[None, 1])
    data_target = tf.placeholder(tf.float32, shape=[None, 1])
    
    net = tflearn.fully_connected(data_input, 128, activation="relu")
    net = tflearn.fully_connected(net, 128, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
#     net = tflearn.fully_connected(net, 32, activation="relu")
    
    data_output = tflearn.fully_connected(net, 1, activation="linear")
    
    loss = tf.reduce_mean(tf.square(data_output - data_target))
    op = optim.minimize(loss)
    
    return op, data_input, data_target, loss, data_output

In [12]:
op, data_x, data_y, loss, mean_grads = bnn.update(8, optim, 1000, 1000)
sess.run(tf.global_variables_initializer())

In [13]:
epoch_loss = []
nn_epoch_loss = []
epochs = 10000

In [14]:
# op, data_x, data_y, loss, data_output = normal_nn(optim)
# sess.run(tf.global_variables_initializer())
# for e in tqdm(range(epochs)):
#     input_batch = data_input
#     target_batch = data_target

#     _, l = sess.run([op, loss], feed_dict={data_x: input_batch, data_y:target_batch})
#     nn_epoch_loss.append(l)

In [15]:
for e in tqdm(range(epochs)):
#     indices = np.random.randint(low=0, high=100, size=batch_size)
#     input_batch = [data_input[i] for i in indices]
#     target_batch = [data_target[i] for i in indices]
    input_batch = data_input
    target_batch = data_target
    
#     grad_nums = [a for (a,b) in grads]
    _, l, mg = sess.run([op, loss, mean_grads], feed_dict={data_x: input_batch, data_y:target_batch})
    epoch_loss.append(l)
    
#     print()
#     print(wg)
#     print()
#     print(mg)
#     print()
#     print(sg)
#     print()
        

#     print("Epoch {}:".format(e))
#     print(sess.run([bnn.qb_mean, bnn.qb_p]))
#     print(g)
#     print()
    #     print("---")
    #     print(sess.run([bnn.qw_p, bnn.qb_p]))    

100%|██████████| 10000/10000 [40:15<00:00,  4.21it/s]


In [16]:
sess.run(bnn.qw_ps[0])

array([[-2.99398351, -3.20956063, -3.02992201, -3.0098381 , -2.97260714,
        -3.33291984, -2.93933201, -2.92901087, -3.19608593, -3.16026402,
        -3.16643643, -3.17071271, -2.94422364, -3.15040445, -3.03308511,
        -2.96808219, -3.08402157, -3.06655359, -3.18688774, -3.34302139,
        -3.03450799, -3.05115652, -3.07682872, -3.18589473, -3.0656271 ,
        -3.13739824, -2.98881626, -3.1380868 , -3.10955834, -2.97471499,
        -2.98040915, -3.15380621, -3.07718968, -3.27897   , -3.07000113,
        -3.20797706, -2.91969872, -3.07875848, -3.11254787, -3.17359447,
        -2.99602747, -3.14348626, -2.93491101, -3.30495358, -3.04114842,
        -3.14662433, -3.11697054, -3.04805231, -2.95364833, -3.05815792,
        -3.06938052, -2.82056499, -3.07349157, -2.89105535, -2.94140148,
        -3.08391333, -3.00491214, -2.89061499, -3.02234387, -3.03038287,
        -2.92344284, -2.94410753, -3.06787181, -3.18295574, -3.27876568,
        -2.8320837 , -3.13271165, -2.96140933, -2.9

In [17]:
test_set = [[x] for x in np.linspace(-0.5,1,1000)]
test_set_flat = [a[0] for a in test_set]
data_input_flat = [a[0] for a in data_input]
data_target_flat = [a[0] for a in data_target]

In [18]:
pred_input, pred_output = bnn.sample()
predictions = sess.run(pred_output, feed_dict={pred_input: test_set})
var_trials = 100
for _ in tqdm(range(var_trials-1)):
    predictions = np.concatenate([predictions, sess.run(pred_output, feed_dict={pred_input: test_set})], axis=1)

100%|██████████| 99/99 [00:00<00:00, 177.12it/s]


In [19]:
pred_means = np.apply_along_axis(lambda x: np.mean(x), 1, predictions)
pred_means[3]

1.5354266

In [20]:
pred_vars = np.apply_along_axis(lambda x: np.std(x, ddof=1), 1, predictions)

In [21]:
# nn_preds = sess.run(data_output, feed_dict={data_x: test_set})

In [22]:
p = figure(width=500, height=500)

err_xs=[]
err_ys=[]
for x, y, yerr in zip(test_set_flat, pred_means, pred_vars):
    err_xs.append((x, x))
    err_ys.append((y - yerr, y + yerr))

p.multi_line(err_xs, err_ys, color=(255,0,0,0.15))

# p.line([a/100 for a in range(100)], [scale*a/100 for a in range(100)], line_width=3, color="blue")
p.line(test_set_flat, pred_means, line_width=2, color="red")

p.line(data_input_flat, data_target_flat, line_width=3, color="blue")

# p.line(test_set_flat, nn_preds.flatten(), line_width=2, color="orange")

# Loss
# p.line(range(epochs), epoch_loss, line_width=2, color="green")

show(p)

In [23]:
loss_p = figure(width=500, height=500)

# Loss
loss_p.line(range(len(epoch_loss)), epoch_loss, line_width=2, color="green")

# loss_p.line(range(len(nn_epoch_loss)), nn_epoch_loss, line_width=2, color="purple")

show(loss_p)